In [8]:
'''
Goal: Develop an algorithm that can estimate the purpose of a satellite from only a limited amount of information.
'''

'''
Data exploration
- Union of Concerned Scientists Satellite Database downloaded Apr 2024 (https://www.ucsusa.org/resources/satellite-database)
'''

# Import the dataset

import pandas as pd
import numpy as np

data = pd.read_csv(r'..\data\UCS-Satellite-Database-5-1-2023.csv')
display(data)


,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,...,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,1HOPSAT-TD (1st-generation High Optical Perfor...,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAC AIS-Sat1 (Kelpie 1),AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalto-1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAt-4,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ABS-2 (Koreasat-8, ST-3)",ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,Ziyuan 1-02C,Ziyuan 1-02C,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7556,Ziyuan 1-2D,Ziyuan 1-2D,NR,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Hyperspectral Imaging,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7557,Ziyuan 3 (ZY-3),Ziyuan 3,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7558,Ziyuan 3-2,Ziyuan 3-2,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Clean the dataset

## Remove all columns that have 'unnamed' in their title
string_to_remove = 'Unnamed'

def remove_unnamed_cols(df, string_to_remove):
    """
    This function removes columns containing a specific string from a pandas DataFrame.

    Args:
        df: The pandas DataFrame to modify.
        string_to_remove: The string to look for in column names.

    Returns:
        A new pandas DataFrame with columns containing the specified string removed.
    """
    return df[[col for col in df.columns if string_to_remove not in col]]

data_cols_removed = remove_unnamed_cols(data.copy(), string_to_remove)
display(data_cols_removed)

,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,...,NORAD Number,Comments,Source Used for Orbital Data,Source,Source.1,Source.2,Source.3,Source.4,Source.5,Source.6
0,1HOPSAT-TD (1st-generation High Optical Perfor...,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,...,44859,Pathfinder for planned earth observation const...,JMSatcat/3_20,https://spaceflightnow.com/2019/12/11/indias-5...,https://www.herasys.com/,NaN,NaN,NaN,NaN,NaN
1,AAC AIS-Sat1 (Kelpie 1),AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,55107,Provide AIS information to Orbcomm.,JMSatcat/9_23,https://www.aac-clyde.space/articles/aac-clyde...,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalto-1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,...,42775,Technology development and education.,JMSatcat/10_17,https://directory.eoportal.org/web/eoportal/sa...,NaN,http://www.planet4589.org/space/log/satcat.txt,NaN,NaN,NaN,NaN
3,AAt-4,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,41460,Carries AIS system.,Space50,http://spaceflightnow.com/2016/04/26/soyuz-bla...,NaN,http://space50.org/objekt.php?mot=2016-025E&ja...,NaN,NaN,NaN,NaN
4,"ABS-2 (Koreasat-8, ST-3)",ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,39508,"32 C-band, 51 Ku-band, and 6 Ka-band transpond...",ZARYA,http://www.absatellite.net/satellite-fleet/?sa...,NaN,http://www.zarya.info/Diaries/Launches/Launche...,http://www.absatellite.net/2010/10/13/asia-bro...,http://www.spacenews.com/article/satellite-tel...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,Ziyuan 1-02C,Ziyuan 1-02C,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,38038,Can acquire high-resolution data through remot...,www.spacedebris.net 12/12,http://www.spaceflightnow.com/news/n1201/09lon...,NaN,https://spacenews.com/china-launches-five-comm...,NaN,NaN,NaN,NaN
7556,Ziyuan 1-2D,Ziyuan 1-2D,NR,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Hyperspectral Imaging,LEO,Sun-Synchronous,...,44528,Hyperspectral imaging,Space50,http://spaceflightnow.com/2016/05/31/long-marc...,NaN,https://www.planet4589.org/space/log/satcat.txt,http://space50.org/objekt.php?mot=2016-033A&ja...,NaN,NaN,NaN
7557,Ziyuan 3 (ZY-3),Ziyuan 3,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,38046,Land survey satellite.,ZARYA,https://spaceflightnow.com/2020/07/25/china-la...,NaN,http://www.lib.cas.cz/space.40/2011/079A.HTM,NaN,NaN,NaN,NaN
7558,Ziyuan 3-2,Ziyuan 3-2,China,China,China Centre for Resources Satellite Data and ...,Government,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,...,41556,Thought to be for intelligence gathering.,JMSatcat/1_22,https://spaceflightnow.com/2021/11/15/japanese...,NaN,https://www.planet4589.org/space/log/satcat.txt,NaN,NaN,NaN,NaN


In [12]:
'''
Selecting features (columns)
- Country of Operator/Owner
- Operator/Owner
- Users
- Class of Orbit
- Type of Orbit
- Contractor
'''

# Remove all other columns except for "Purpose" which will be the target (label)
columns_to_keep = ['Country of Operator/Owner', 'Operator/Owner', 'Users', 'Class of Orbit', 'Type of Orbit', 'Contractor', 'Purpose']

data_features_kept = data_cols_removed[columns_to_keep]
display(data_features_kept)

,Country of Operator/Owner,Operator/Owner,Users,Class of Orbit,Type of Orbit,Contractor,Purpose
0,USA,Hera Systems,Commercial,LEO,Non-Polar Inclined,Hera Systems,Earth Observation
1,United Kingdom,AAC Clyde Space,Commercial,LEO,Sun-Synchronous,AAC Clyde Space,Earth Observation
2,Finland,Aalto University,Civil,LEO,Sun-Synchronous,Aalto University,Technology Development
3,Denmark,University of Aalborg,Civil,LEO,Sun-Synchronous,University of Aalborg,Earth Observation
4,Multinational,Asia Broadcast Satellite Ltd.,Commercial,GEO,NaN,Space Systems/Loral,Communications
...,...,...,...,...,...,...,...
7555,China,China Centre for Resources Satellite Data and ...,Government,LEO,Sun-Synchronous,China Academy of Space Technology (CAST),Earth Observation
7556,China,China Centre for Resources Satellite Data and ...,Government,LEO,Sun-Synchronous,China Academy of Space Technology (CAST),Earth Observation
7557,China,China Centre for Resources Satellite Data and ...,Government,LEO,Sun-Synchronous,China Academy of Space Technology (CAST),Earth Observation
7558,China,China Centre for Resources Satellite Data and ...,Government,LEO,Sun-Synchronous,China Academy of Space Technology (CAST),Earth Observation
